In [2]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os

In [60]:
def load_house_attributes(inputPath):
    cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
    df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)
    #key値の方をリストにする
    zipcodes = df["zipcode"].value_counts().keys().tolist()
    #カウント数の方をリストにする
    counts = df["zipcode"].value_counts().tolist()
    print(counts)
    
    for (zipcode, count) in zip(zipcodes, counts):
        if count < 25:
            #zipcodeが一致するインデックスをidxsに格納する
            idxs = df[df["zipcode"] == zipcode].index
            #idxsの値を持つインデックスを削除する
            #inplaceの説明
            #デフォルトでは元のDataFrameは変更されず、新しいDataFrameが返される。
            #引数inplaceをTrueにすると、元のDataFrameが変更される。
            #この場合、新しいDataFrameは返されず、返り値はNone。
            df.drop(idxs, inplace=True)
            
    return df

def preprocess_house_attributes(df, train, test):
    continuous = ["bedrooms", "bathrooms", "area"]
    
    #最小が0、最大が1になるようにする
    cs = MinMaxScaler()
    trainContinuous = cs.fit_transform(train[continuous])
    print(trainContinuous.shape)
    testContinuous = cs.transform(test[continuous])
    print(testContinuous.shape)
    
    zipBinarizer = LabelBinarizer().fit(df["zipcode"])
    #zipcodeがcount数以上の数だけカテゴリを作り、それぞれのuniqueな
    #zipcodeに対してone-hotエンコーディングを行う
    trainCategorical = zipBinarizer.transform(train["zipcode"])
    print(trainCategorical.shape)
    #(指定のカウント数によるデータの制限はtrain_test_splitに行ったため、
    #テスト用のカテゴリも学習用に一致する)
    testCategorical = zipBinarizer.transform(test["zipcode"])
    print(testCategorical.shape)
    
    trainX = np.hstack([trainCategorical, trainContinuous])
    testX = np.hstack([testCategorical, testContinuous])
    
    return (trainX, TestX)

def load_house_image(df, inputPath):
    images = []
    
    for i in df.index.values:
        basePath = os.path.sep.join([inputPath, "{}_".format(i + 1)])

In [61]:
from sklearn.model_selection import train_test_split

df = load_house_attributes("../Houses-dataset/Houses Dataset/HousesInfo.txt")
(train, test) = train_test_split(df, test_size=0.25, random_state=42)
preprocess_house_attributes(df, train, test)


[100, 60, 54, 49, 41, 32, 26, 22, 12, 12, 11, 11, 11, 11, 10, 9, 9, 7, 4, 4, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(271, 3)
(91, 3)
(271, 7)
(91, 7)


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
